In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ["GOOGLE_API_KEY"] = ""

llm = ChatGoogleGenerativeAI(model = "gemini-pro")

In [4]:
from crewai import Crew,Agent,Task

In [5]:
conversation = """
Hi Sneh, how are you feeling today?

Hi, Doctor. I still have a high fever, cold, and cough. It's been like this for the past two days.

I see. Based on the symptoms you've described—high fever, cold, and cough—it sounds like you might have a viral infection, such as the flu or a common cold. These infections are quite common and usually resolve on their own with proper rest and medication. However, it’s important to monitor the fever and ensure it doesn’t get too high.

That makes sense. What can I do to feel better?

Continue taking Paracetamol for the fever. It's good to hear that it has been effective for you. I'll prescribe a few more medications to help with your symptoms. Take Paracetamol 500mg every 6 hours as needed for fever, Cetirizine 10mg before bed for your cold symptoms, and Guaifenesin 200mg after breakfast and after dinner for the cough.

Make sure to drink plenty of fluids and get plenty of rest.

Got it. How long should I continue taking these medications?

Take the medications for the next three days. If your symptoms persist or worsen, come back sooner. Otherwise, I would like you to come back for a follow-up check-up in five days to make sure you are recovering well.

Okay, I will do that. Thank you, Doctor.

You're welcome, Sneh. Take care and get well soon. Don't hesitate to reach out if you have any concerns before your follow-up.
"""

In [7]:
general_information = "Patient Name: Sneh Age: 20 Symptoms: - High fever (99 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None"

In [23]:
from crewai import Crew, Agent, Task


llm_agent = Agent(
    role="Patient Support Agent",
    goal="To provide post-medical consultation support to patients.",
    backstory="Trained to assist patients in managing their health after medical consultations.",
    llm=llm,
    tools=[], 
    allow_delegation=False,
    verbose=True
)

# Define the task for the LLM agent
support_task = Task(
    description=(
        "Here is patient general information: {general_information}\n"
        "Here is the conversation between the patient and the doctor: {conversation}\n\n"
        "Provide recommendations for recovery, medication details, and schedule a follow-up check-up based on the patient details and conversation between patient and doctor."
        "After you get the necessary information parse it in json format where data should be in plain text."
        "Only return the json data with no additional text"
    ),
    expected_output="Structured advice for the patient including recovery recommendations, medication details, and follow-up date.",
    agent=llm_agent
)



In [24]:
crew = Crew(agents=[llm_agent], tasks=[support_task],verbose=True)
inputs = {"general_information": general_information, "conversation": conversation}

2024-06-26 18:10:14,324 - 11752 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
result = crew.kickoff(inputs)

 [2024-06-26 18:10:15][DEBUG]: == Working Agent: Patient Support Agent
 [2024-06-26 18:10:15][INFO]: == Starting Task: Here is patient general information: Patient Name: Sneh Age: 20 Symptoms: - High fever (99 degrees Fahrenheit) - Cold - Cough Pain Assessment: None provided Medication: Paracetamol, effective for a few hours Symptom Duration: 2 days Additional Information: None
Here is the conversation between the patient and the doctor: 
Hi Sneh, how are you feeling today?

Hi, Doctor. I still have a high fever, cold, and cough. It's been like this for the past two days.

I see. Based on the symptoms you've described—high fever, cold, and cough—it sounds like you might have a viral infection, such as the flu or a common cold. These infections are quite common and usually resolve on their own with proper rest and medication. However, it’s important to monitor the fever and ensure it doesn’t get too high.

That makes sense. What can I do to feel better?

Continue taking Paracetamol for 

In [26]:
print(result)

```json
{
  "recovery_recommendations": [
    "Get plenty of rest.",
    "Drink plenty of fluids."
  ],
  "medication_details": [
    {
      "name": "Paracetamol",
      "dosage": "500mg",
      "frequency": "every 6 hours as needed for fever"
    },
    {
      "name": "Cetirizine",
      "dosage": "10mg",
      "frequency": "before bed for cold symptoms"
    },
    {
      "name": "Guaifenesin",
      "dosage": "200mg",
      "frequency": "after breakfast and after dinner for cough"
    }
  ],
  "follow_up_details": {
    "date": "in five days",
    "purpose": "to make sure you are recovering well"
  }
}
```
